In [4]:
import os
# from google.colab import drive
# from os.path import join as jn

# # 구글 드라이브 접근
# ROOT = "/content/drive"
# try:
#   drive.mount(ROOT, force_remount=True)
# except:
#   drive.mount(ROOT)
ROOT = os.getcwd()

import json
import tqdm as root_tqdm
from tqdm import tqdm
from glob import glob

import numpy as np
import cv2 as cv
from PIL import Image

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import pycocotools.coco as pycocos
from pycocotools.coco import COCO

import torch
import torch.optim as optim
from torch  import nn, Tensor

import torchvision
# 본인 경로에 맞게 수정하면 됨.
# PATH = jn(ROOT, "MyDrive/Task/plastic-segmentation/Data")
# PATH = jn(ROOT, "MyDrive/Task/plastic-segmentation/Sample_data")
PATH = os.path.join(ROOT, "assets/data")
# # 구글 드라이브 경로에서 '/content/sample_data/data"로 복사 시
# import shutil
# shutil.copytree(PATH +"/train", "./sample_data/data/train")


basic_packages = [json, root_tqdm, np, cv, Image, mpl, pycocos, torch, torchvision]
print("Packages Version Infos :")
for pname in basic_packages:
    try:
        print(f"\t{pname.__name__}\t:\t{pname.__version__}")
    except:
        print(f"\tmodule '{pname.__name__}' has no attribute '__version__'")

Packages Version Infos :
	json	:	2.0.9
	tqdm	:	4.62.3
	numpy	:	1.21.1
	cv2.cv2	:	4.5.3
	PIL.Image	:	8.4.0
	matplotlib	:	3.5.0
	pycocotools.coco	:	2.0
	torch	:	1.10.0
	torchvision	:	0.11.1


# general flow

* Dataset
    * CustomDataset
    * DataLoaader
* Model
    * CustomModel
    * train
    * inference
* VSL
    * predict(inference)
    * output

In [28]:
# Mask RCNN 공식 홈페이지 Tutorial을 사용하여, 필요한 코드를 불러옵니다.
# 해당 튜토리얼과 동일하게 상위 폴더로 가져와도 되고, 특정 폴더로 옮겨도 되지만 일단은 py를 ipynb에서 읽을 수 있도록 호출
%load_ext autoreload
%autoreload 2

# check shape of datasets("PennFudanPed")

* tutorial까지 돌아간 코드에서 shape 확인하고 customize...근데 사용한 metric?module?이 다른 걸 보면 아예 데이터 입력 방식이 다른 것일 수도...

* torch.uint8 -> torch.ByteTensor
* torch.float16 -> torch.HalfTensor
* torch.float, torch.float32 -> torch.FloatTensor
* torch.int16 -> torch.ShortTensor
* torch.int, torch.int32 -> torch.IntTensor
* torch.int64 -> torch.LongTensor
* torch.float64 -> torch.DoubleTensor

In [81]:
# print(f"0번째 train 이미지")
# print(f"\t 이미지 shape : {dataset[0][0].shape}\n\t 이미지 type : {dataset[0][0].type()}")
# for k in list(dataset[0][1].keys()):
#      print(f"\t\tdict_keys == '{k}' :  \n\t\t\tshape : {dataset[0][1][k].shape}\n\t\t\ttype : {dataset[0][1][k].type()}")
#      if k is not "masks":
#          print(f"\t\t\t{dataset[0][1][k]}")

0번째 train 이미지
	 이미지 shape : torch.Size([3, 321, 398])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([1, 4])
			type : torch.FloatTensor
			tensor([[170.,  21., 319., 311.]])
		dict_keys == 'labels' :  
			shape : torch.Size([1])
			type : torch.LongTensor
			tensor([1])
		dict_keys == 'masks' :  
			shape : torch.Size([1, 321, 398])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([1])
			type : torch.LongTensor
			tensor([155])
		dict_keys == 'area' :  
			shape : torch.Size([1])
			type : torch.FloatTensor
			tensor([43210.])
		dict_keys == 'iscrowd' :  
			shape : torch.Size([1])
			type : torch.LongTensor
			tensor([0])


masks를 확인하고, 다른 값들을 확인한 결과,,
    * mask 생성 무조건 해야함!!!
    * 형태(모델을 돌리는 방법)는 자유겠지만, 그 각각을 맞추려면 CustomDataset(default)에서 CustomDataset(multi class or process)로 진행되어야 함.

* DataLoader

operation이 어떻게 작용되는지 확인하기

# check sorting path process

```python
# author : qkrwjdduf159 <pde159@naver.com>
# version 1
## train
trn_path = f"{PATH}/train/annotation"
file_list = os.listdir(trn_path)

train_files = []
for file in tqdm(file_list):
    dir = trn_path + '/' + file
    json_list = glob(dir + '/' + '*.json')
    train_files.append(json_list)

train_json_list = []
for files in tqdm(train_files):
    for json_file in tqdm(files):
        train_json_list.append(json_file)

## test
tst_PATH = f"{PATH}/test/annotations"
file_list = os.listdir(tst_PATH)

test_files = []
for file in tqdm(file_list):
    dir = tst_PATH + '/' + file
    json_list = glob(dir + '/' + '*.json')
    test_files.append(json_list)

test_json_list = []
for files in tqdm(test_files):
    for json_file in tqdm(files):
        test_json_list.append(json_file)
```


```python
# author : AshbeeKim <ksb.forest@gmail.com>
# version 2
trn = os.path.join(PATH, "train")
tst = os.path.join(PATH, "test")

baseDF = {"kind" : [], "label" : [], "metainfo_id" : [], "feature" : [], "image_path" : [], "annot_path": []}
trn_num, tst_num = 0, 0
for fpath in tqdm([trn, tst]):
    kind = os.path.basename(fpath)
    BDIR = sorted(os.listdir(fpath))    # image, annotation(s)
    for bdir in BDIR:
        for dirs in sorted(os.listdir(jn(fpath, bdir))):
            dpath = jn(fpath, bdir, dirs)
            if bdir.lower()=='image':
                paths = sorted(glob(dpath + "/*.jpg"))
                baseDF['image_path'].extend(paths)
                kinds = [kind for cnt in range(len(paths))]
                baseDF['kind'].extend(kinds)
                if kind=='train' : trn_num += len(paths)
                elif kind=='test' : tst_num += len(paths)
                fnames = [os.path.basename(fname) for fname in paths]
                labels = list(map(lambda x: (x.split('_')[0]), fnames))
                baseDF['label'].extend(labels)
                metaIds = list(map(lambda x: int(x.split('_')[1]), fnames))
                baseDF['metainfo_id'].extend(metaIds)
                feats = list(map(lambda x: int(x.split('_')[-1][:-4]), fnames))
                baseDF['feature'].extend(feats)
            else:
                paths = sorted(glob(dpath + "/*.json"))
                baseDF['annot_path'].extend(paths)
```

## Fix Seeds

In [15]:
# cotributed by @qkrwjdduf156, @AshbeeKim
# import libraries

import os
import random
import time
import json
from glob import glob
# from tqdm.notebook import tqdm
from tqdm import tqdm

import collections
import numpy as np
import pandas as pd

import cv2 as cv
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torchvision
from torchvision.transforms import ToPILImage
from torchvision.transforms import functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor

In [16]:
# Fix randomness
# * ref : https://www.kaggle.com/rluethy/sartorius-torch-mask-r-cnn

def fix_all_seeds(seed):
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)
        torch.backends.cudnn.deterministic = True
    
fix_all_seeds(513610)

data_directory = PATH
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
BATCH_SIZE = 1
NUM_EPOCHS = 5

# Reduced the train dataset to 5000 rows
TEST = False

# Dataset

masks를 확인하고, 다른 값들을 확인한 결과,, 
* mask 생성 무조건 해야함!!! 
* 형태(모델을 돌리는 방법)는 자유겠지만, 그 각각을 맞추려면 CustomDataset(default)에서 CustomDataset(multi class or process)로 진행되어야 함.

```python
0번째 train 이미지
	 이미지 shape : torch.Size([3, 321, 398])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([1, 4])
			type : torch.FloatTensor
			tensor([[170.,  21., 319., 311.]])
		dict_keys == 'labels' :  
			shape : torch.Size([1])
			type : torch.LongTensor
			tensor([1])
		dict_keys == 'masks' :  
			shape : torch.Size([1, 321, 398])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([1])
			type : torch.LongTensor
			tensor([155])
		dict_keys == 'area' :  
			shape : torch.Size([1])
			type : torch.FloatTensor
			tensor([43210.])
		dict_keys == 'iscrowd' :  
			shape : torch.Size([1])
			type : torch.LongTensor
			tensor([0])

# torch.uint8 -> torch.ByteTensor
# torch.float16 -> torch.HalfTensor
# torch.float, torch.float32 -> torch.FloatTensor
# torch.int16 -> torch.ShortTensor
# torch.int, torch.int32 -> torch.IntTensor
# torch.int64 -> torch.LongTensor
# torch.float64 -> torch.DoubleTensor

# list -> tensor ; torch.as_tensor
# list
```

## CustomDataset

image segmentation을 위해선 target[boxes, labels, masks] 필수
- 통상 "image_id", 마스크가 속한 이미지 id
- "area", box 면적, pred box, act box IoU 구할 때 필요
- "iscrowd", 작은 물체를 하나의 군집으로 박스처리하여 레이블링 했는지에 관한 여부, 물체가 숨어져있거나, 가려진 경우 사용

In [84]:
from torch.utils.data import Dataset


class CustomDataset(Dataset):
    def __init__(self, root, transforms = None, target_transforms = None, mode = 'train'):
        self.root = root
        self.transforms = transforms
        self.target_transforms = target_transforms
        self.mode = mode.lower()
        self.infos = self.baseInfos()
        self.imgs, self.annots = self.infos['image_path'], self.infos['annot_path']
        # if self.mode == 'train':
        #     self.imgs, self.masks = blahblah
        self.masks = None

    def __getitem__(self, idx=None):
        '''return tuple(image, target)'''
        annot_path = self.annots[idx]
        mask_shape, target = self.json2annots(annot_path, idx)
        # target : dict

        image_path = self.imgs[idx]
        img, background = self.imageNmask(image_path, mask_shape)
        
        if self.mode == "train":
            # train인 경우에만 target['masks'] 처리할 함수가 들어가야 함.
            target['masks'] = self.createMask(background, target['masks'])

        if self.transforms is not None:
            try:
                data = self.transforms(img, target)
                return data
            except:
                if self.transforms is not None:
                    img = self.transforms(img)
                elif self.target_transforms is not None:
                    target = self.target_transforms(target)
                return img, target
        else:
            img = self.preprocess(img)
            return img, target

    def baseInfos(self):
        baseinfos = {"label" : [], "metainfo_id" : [], "feature" : [], "image_path" : [], "annot_path": []}
        
        bpath = os.path.join(self.root, self.mode)
        bdirs = sorted(os.listdir(bpath))   # image, annotation(s)
        
        for bdir in tqdm(bdirs):
            for dirs in sorted(os.listdir(os.path.join(bpath, bdir))):
                dpath = os.path.join(bpath, bdir, dirs)
                
                if bdir.lower()=='image':
                    paths = sorted(glob(dpath + "/*.jpg"))
                    baseinfos['image_path'].extend(paths)
                    fnames = [os.path.basename(fname) for fname in paths]
                    labels = list(map(lambda x: (x.split('_')[0]), fnames))
                    baseinfos['label'].extend(labels)
                    metaIds = list(map(lambda x: int(x.split('_')[1]), fnames))
                    baseinfos['metainfo_id'].extend(metaIds)
                    feats = list(map(lambda x: int(x.split('_')[-1][:-4]), fnames))
                    baseinfos['feature'].extend(feats)
                else:
                    paths = sorted(glob(dpath + "/*.json"))
                    baseinfos['annot_path'].extend(paths)
        return baseinfos
    
    def imageNmask(self, image_path, mask_shape):
        image = cv.imread(image_path)
        image = cv.cvtColor(image, cv.COLOR_BGR2RGB)    # [H, W, C]
        # mask = np.zeros(image.shape, dtype=np.uint8)    # [H, W, C] >> [N+1, H, W, C] >> [N, H, W]
        masks = np.zeros(mask_shape, dtype = np.uint8)  
        # 굳이 3채널 정보가 필요하지 않다면, (H, W) 만 받아도, instance 공간 할당은 이미 해놓음
        # mask 생성 함수보고 고쳐야 할 듯
        image = Image.fromarray(image)  # cv to PIL
        # image = torchvision.transforms(image)    # [C, H, W] 
        return image, masks

    def preprocess(self, img):
        image = img
        import torchvision.transforms as T
        m, s = np.mean(image, axis = (0, 1)), np.std(image, axis = (0, 1))        
        if self.mode == 'train':
            transform = T.Compose([
                                   T.ToTensor(),
                                   T.Normalize(mean = m, std = s),
            ])
            image = transform(image)
        else:
            transform = T.Compose([
                                   T.Resize(256),
                                   T.ToTensor(),
                                   T.Normalize(mean = m, std = s),
            ])
            image = transform(image)
        return image
    
    def json2annots(self, annot_path, idx):
        with open(annot_path, 'r') as f:
            annot = json.loads(f.read())

        n_objects = len(annot['annotations'])
        W, H = int(annot['images'][0]['width']), int(annot['images'][0]['height'])

        if self.mode == 'train':
            # target = {"boxes" : [], "labels" : [], "masks" : [np.zeros((W, H), dtype = np.uint8)], "image_id" : [], "area" : [], "iscrowd" : []}
            target = {"boxes" : [], "labels" : [], "masks" : [], "image_id" : [], "area" : [], "iscrowd" : []}

            # bbox ground bbox
            gxmin, gymin, gxmax, gymax = 0, 0, 0, 0
            for i in range(n_objects):
                bbox = annot['annotations'][i]['bbox']
                xmin, ymin, width, height = bbox[0],bbox[1],bbox[2],bbox[3]
                xmax, ymax = xmin + width, ymin + height
                target['boxes'].append([xmin, ymin, xmax, ymax])
                # ########## just-in-case(ground_bbox) ##########
                # gxin = xmin if xmin > gxmin else gxmin
                # gymin = ymin if ymin > gymin else gymin
                # gmax = xmax if xmax > gxmax else gxmax
                # gmax = ymax if ymax > gymax else gymax
                # target['boxes'].append([xmin, ymin, xmax, ymax])

                label = annot['annotations'][i]['category_id']
                # torch.ones로 하지 않은 이유는 클래스 정보가 들어가야된다고 판단했기 때문
                target['labels'].append([label])

                target['masks'].append(np.zeros((W, H), dtype=np.uint8))
                # mask 처리 전, 공간 할당

                area = torch.tensor(annot['annotations'][i]['area'], dtype = torch.float32)
                target['area'].append([area])

                iscrowd = torch.tensor(annot['annotations'][i]['iscrowd'], dtype = torch.int64)
                target['iscrowd'].append([iscrowd])
                # ignore = torch.tensor(annot['annotations'][i]['ignore'], dtype = torch.int64)
                # target['ignore'].append([ignore])

            ########################################################## instance id, 필요 시 주석해제 ##########################################################    
            # segmentation에 DataLoader 내에서 어떻게 선언되어 있는지 확인해야 함
            # target['id'] = torch.stack([torch.tensor([i+1]) for i in range(n_objects)]) # shape : (3, 1)
            # ValueError: Expected target boxes to be a tensorof shape [N, 4], got torch.Size([1, 3, 4]).
            ########################################################## type은 맞으나, shape이 맞는지 확신이 없는 부분 ##########################################################
            # torch.Size([3, 2048, 2048])
            # target['boxes'] = torch.as_tensor(target['boxes'], dtype = torch.float32)   
            # shape : (3, 4)
            target['boxes'] = torch.stack([torch.tensor(bbox, dtype = torch.float32) for bbox in target['boxes']])

            # shape : torch.Size([1, 3, 1])
            target['labels'] = torch.as_tensor((target['labels'], ), dtype = torch.int64).squeeze()
            
            # shape : torch.Size([1, 4, 2048, 2048])
            target['masks'] = torch.as_tensor(target['masks'], dtype = torch.uint8)
            # mssk 처리 전, 공간 할당

            ########################################################## below ~~ torch.Size([3, 1]) : 각각의 shape도 type도 맞는 것 같음 ##########################################################
            # image_path[index] -> 이미지 자체의 아이디
            target['image_id'] = torch.stack([torch.tensor([idx]) for i in range(n_objects)])   # shape : (3, 1)
            target['area'] = torch.as_tensor(target['area'], dtype = torch.float32) # shape : (1, 3) >> (3, 1)
            target['iscrowd'] = torch.as_tensor(target['iscrowd'], dtype = torch.int64)
            # target['ignore'] = torch.as_tensor(target['ignore'], dtype = torch.int64)



        else:
            target = {}
            try:
                target['image_id'] = torch.stack([torch.tensor([idx]) for i in range(n_objects)], dtype = torch.int64)
            except:
                target['image_id'] = torch.tensor([idx], dtype=torch.int)
        return (n_objects, W, H), target

    def createMask(self, background, mask):
        # bbox

        # segmentation

        return mask

    
    def _labels2category(self, label):
        category = {1 : "PET", 2 : "PS", 3 : "PP", 4 : "PE"}
        return category(label)

    def __len__(self):
        return len(self.imgs)

    # ########## just-in-case ##########
    # # when we need more edas
    # def additional_infos(self):
    #     with open(self.annots[0], 'r') as f:
    #         annot = json.loads(f.read())
    #     metainfo = annot['metainfo']

In [85]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

dataset = CustomDataset(PATH)

idx = random.randint(0, dataset.__len__())

print(f"{idx}번째 train 이미지")

# C, W, H, N
# 
# num_classes, W, H, Batch(num_instance + 1) : mask
print(f"\t 이미지 shape : {dataset[idx][0].shape}\n\t 이미지 type : {dataset[idx][0].type()}")
for k in list(dataset[idx][1].keys()):
     print(f"\t\tdict_keys == '{k}' :  \n\t\t\tshape : {dataset[idx][1][k].shape}\n\t\t\ttype : {dataset[idx][1][k].type()}")
     if k is not "masks":
         print(f"\t\t\t{dataset[idx][1][k]}")

100%|██████████| 2/2 [00:00<00:00, 18.69it/s]


2142번째 train 이미지
	 이미지 shape : torch.Size([3, 2048, 2048])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([3, 4])
			type : torch.FloatTensor
			tensor([[ 824.,  457., 1204.,  852.],
        [ 394.,  979.,  778., 1364.],
        [1009., 1128., 1354., 1469.]])
		dict_keys == 'labels' :  
			shape : torch.Size([3])
			type : torch.LongTensor
			tensor([3, 3, 3])
		dict_keys == 'masks' :  
			shape : torch.Size([3, 2048, 2048])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[2142],
        [2142],
        [2142]])
		dict_keys == 'area' :  
			shape : torch.Size([3, 1])
			type : torch.FloatTensor
			tensor([[117983.5000],
        [116196.5000],
        [ 91739.0000]])
		dict_keys == 'iscrowd' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[0],
        [0],
        [0]])


In [86]:
%timeit
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

dataset = CustomDataset(PATH)

idx = random.randint(0, dataset.__len__())

print(f"{idx}번째 train 이미지")

# C, W, H, N
# 
# num_classes, W, H, Batch(num_instance + 1) : mask
print(f"\t 이미지 shape : {dataset[idx][0].shape}\n\t 이미지 type : {dataset[idx][0].type()}")
for k in list(dataset[idx][1].keys()):
     print(f"\t\tdict_keys == '{k}' :  \n\t\t\tshape : {dataset[idx][1][k].shape}\n\t\t\ttype : {dataset[idx][1][k].type()}")
     if k is not "masks":
         print(f"\t\t\t{dataset[idx][1][k]}")

100%|██████████| 2/2 [00:00<00:00, 19.47it/s]


2998번째 train 이미지
	 이미지 shape : torch.Size([3, 2048, 2048])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([2, 4])
			type : torch.FloatTensor
			tensor([[ 425.,  638.,  902., 1261.],
        [1278.,  538., 1670., 1218.]])
		dict_keys == 'labels' :  
			shape : torch.Size([2])
			type : torch.LongTensor
			tensor([3, 3])
		dict_keys == 'masks' :  
			shape : torch.Size([2, 2048, 2048])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([2, 1])
			type : torch.LongTensor
			tensor([[2998],
        [2998]])
		dict_keys == 'area' :  
			shape : torch.Size([2, 1])
			type : torch.FloatTensor
			tensor([[172804.],
        [220773.]])
		dict_keys == 'iscrowd' :  
			shape : torch.Size([2, 1])
			type : torch.LongTensor
			tensor([[0],
        [0]])


## DataLoader

In [100]:
from engine import train_one_epoch, evaluate
import utils
import transforms as T

data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=utils.collate_fn)

# train
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections

# inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

In [101]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

dataset = CustomDataset(PATH)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=2, shuffle=True, collate_fn=utils.collate_fn)

# train
images,targets = next(iter(data_loader))
images = list(image for image in images)
targets = [{k: v for k, v in t.items()} for t in targets]
output = model(images,targets)   # Returns losses and detections

# inference
model.eval()
x = [torch.rand(3, 300, 400), torch.rand(3, 500, 400)]
predictions = model(x)           # Returns predictions

# /usr/local/lib/python3.7/dist-packages/torch/functional.py:445: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2157.)
#   return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

100%|██████████| 2/2 [00:00<00:00, 18.32it/s]


In [102]:
predictions

[{'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([], grad_fn=<IndexBackward0>)},
 {'boxes': tensor([], size=(0, 4), grad_fn=<StackBackward0>),
  'labels': tensor([], dtype=torch.int64),
  'scores': tensor([], grad_fn=<IndexBackward0>)}]

# Model

In [182]:
import torch
from torch import nn
import torchvision

def quick_check(dataset, idx):
    print(f"{idx}번째 {dataset.mode} 이미지")
    print(f"\t 이미지 shape : {dataset[idx][0].shape}\n\t 이미지 type : {dataset[idx][0].type()}")
    for k in list(dataset[idx][1].keys()):
        print(f"\t\tdict_keys == '{k}' :  \n\t\t\tshape : {dataset[idx][1][k].shape}\n\t\t\ttype : {dataset[idx][1][k].type()}")
        if k is not "masks":
            print(f"\t\t\t{dataset[idx][1][k]}\n\n")

def uni_inference(precision, idx):
    print(f"inference no.{idx} : ")
    for k in list(precision[0].keys()):
        print(f"\t{k} : \n\t\t shape : {precision[0][k].shape}\n")

# fastrcnn
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

# unet ; torch.hub

# fasterrcnn
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

# %%writefile ./CaliberDetector.py    # customize 끝내고 run

# detection
from collections import OrderedDict
from torch import nn
from torchvision.ops import MultiScaleRoIAlign
from torchvision._internally_replaced_utils import load_state_dict_from_url
from torchvision.models.detection._utils import overwrite_eps
from torchvision.models.detection.faster_rcnn import FasterRCNN
from torchvision.models.detection.backbone_utils import resnet_fpn_backbone, _validate_trainable_layers

## briefly check models

### * fastrcnn

In [103]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor

model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained = True)

num_classes = 5 # number_of_class + background
in_features = model.roi_heads.box_predictor.cls_score.in_features

model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

In [104]:
model.eval()

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(

In [107]:
dataset = CustomDataset(PATH)

idx = random.randint(0, dataset.__len__())

print(f"\n\n{idx}번째 train 이미지")

# C, W, H, N
# 
# num_classes, W, H, Batch(num_instance + 1) : mask
print(f"\t 이미지 shape : {dataset[idx][0].shape}\n\t 이미지 type : {dataset[idx][0].type()}")
for k in list(dataset[idx][1].keys()):
     print(f"\t\tdict_keys == '{k}' :  \n\t\t\tshape : {dataset[idx][1][k].shape}\n\t\t\ttype : {dataset[idx][1][k].type()}")
     if k is not "masks":
         print(f"\t\t\t{dataset[idx][1][k]}")

img, _ = dataset.__getitem__(idx)
input_batch = img.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

precision = model(input_batch)

100%|██████████| 2/2 [00:00<00:00, 17.92it/s]


416번째 train 이미지
	 이미지 shape : torch.Size([3, 2048, 2048])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([3, 4])
			type : torch.FloatTensor
			tensor([[1321.,  953., 1538., 1245.],
        [ 614.,  872.,  845., 1097.],
        [1154.,  438., 1363.,  632.]])
		dict_keys == 'labels' :  
			shape : torch.Size([3])
			type : torch.LongTensor
			tensor([4, 4, 4])
		dict_keys == 'masks' :  
			shape : torch.Size([3, 2048, 2048])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[416],
        [416],
        [416]])
		dict_keys == 'area' :  
			shape : torch.Size([3, 1])
			type : torch.FloatTensor
			tensor([[48670.5000],
        [32862.0000],
        [34117.0000]])
		dict_keys == 'iscrowd' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[0],
        [0],
        [0]])


In [114]:
print(f"\n\nTrain 데이터[{idx}]로 inference한 결과 : ")
for k in list(precision[0].keys()):
    print(f"\t{k} : \n\t\t shape : {precision[0][k].shape}\n\t\t mean : {precision[0][k].mean")

precision[0]['boxes'].shape

Train 데이터[416]로 inference한 결과 : 
	boxes : 
		 shape : torch.Size([100, 4])
	labels : 
		 shape : torch.Size([100])
	scores : 
		 shape : torch.Size([100])


torch.Size([100, 4])

### * unet

In [135]:
import torch
from torch import nn
from torchvision import transforms as T

model = torch.hub.load('mateuszbuda/brain-segmentation-pytorch', 'unet',
    in_channels=3, out_channels=1, init_features=32, pretrained=True)
# model = torch.hub._load_local()   # local dire에서 hubconf에 따라 사용이 가능하다고 함

Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/archive/master.zip" to /root/.cache/torch/hub/master.zip
Downloading: "https://github.com/mateuszbuda/brain-segmentation-pytorch/releases/download/v1.0/unet-e012d006.pt" to /root/.cache/torch/hub/checkpoints/unet-e012d006.pt


In [136]:
model.eval()

UNet(
  (encoder1): Sequential(
    (enc1conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu1): ReLU(inplace=True)
    (enc1conv2): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc1norm2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc1relu2): ReLU(inplace=True)
  )
  (pool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (encoder2): Sequential(
    (enc2conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (enc2relu1): ReLU(inplace=True)
    (enc2conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (enc2norm2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, tra

In [141]:
train_dataset = CustomDataset(PATH, mode='train')
test_dataset = CustomDataset(PATH, mode='test')

train_idx = random.randint(0, train_dataset.__len__())
test_idx = random.randint(0, test_dataset.__len__())

print("\nTrain Datasets")
quick_check(train_dataset, train_idx)
print("\nTest Datasets")
quick_check(test_dataset, test_idx)

100%|██████████| 2/2 [00:00<00:00, 107.60it/s]



Train Datasets
232번째 train 이미지
	 이미지 shape : torch.Size([3, 2048, 2048])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([3, 4])
			type : torch.FloatTensor
			tensor([[1064.,  496., 1285.,  676.],
        [1389.,  897., 1550., 1107.],
        [ 679.,  799.,  835., 1020.]])


		dict_keys == 'labels' :  
			shape : torch.Size([3])
			type : torch.LongTensor
			tensor([4, 4, 4])


		dict_keys == 'masks' :  
			shape : torch.Size([3, 2048, 2048])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[232],
        [232],
        [232]])


		dict_keys == 'area' :  
			shape : torch.Size([3, 1])
			type : torch.FloatTensor
			tensor([[25640.5000],
        [25074.5000],
        [29585.0000]])


		dict_keys == 'iscrowd' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[0],
        [0],
        [0]])



Test Datasets
393번째 test 이미지
	 이미지 shape : torch.Size([3, 256, 256

In [142]:
img, _ = train_dataset.__getitem__(train_idx)
input_batch = img.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

In [143]:
output

tensor([[[[1.6307e-05, 1.5145e-05, 1.5356e-05,  ..., 1.6836e-05,
           1.5657e-05, 1.4536e-05],
          [2.0911e-05, 2.0291e-05, 2.0425e-05,  ..., 1.6934e-05,
           1.1150e-05, 1.5441e-05],
          [1.9988e-05, 3.1703e-05, 2.4566e-05,  ..., 1.7147e-05,
           1.6356e-05, 1.2977e-05],
          ...,
          [2.0111e-05, 2.4908e-05, 2.0760e-05,  ..., 1.2605e-05,
           1.1867e-05, 1.0306e-05],
          [1.9120e-05, 2.7806e-05, 2.3741e-05,  ..., 1.6201e-05,
           1.1835e-05, 1.3853e-05],
          [1.2632e-05, 1.8156e-05, 2.1674e-05,  ..., 1.8309e-05,
           1.4875e-05, 1.0430e-05]]]], device='cuda:0')

In [144]:
print(torch.round(output))

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0')


In [145]:
output.shape

torch.Size([1, 1, 2048, 2048])

In [157]:
img, _ = test_dataset.__getitem__(test_idx)

input_batch = img.unsqueeze(0)

model.eval()
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

confidences = model(input_batch)

In [158]:
confidences

tensor([[[[1.5116e-05, 1.5655e-05, 1.6071e-05,  ..., 1.5135e-05,
           1.5036e-05, 1.6059e-05],
          [1.4148e-05, 1.7100e-05, 1.7359e-05,  ..., 1.8232e-05,
           1.0345e-05, 1.6023e-05],
          [1.2134e-05, 1.9303e-05, 1.4963e-05,  ..., 1.4009e-05,
           1.1067e-05, 1.2937e-05],
          ...,
          [1.3299e-05, 1.9279e-05, 1.4391e-05,  ..., 1.0745e-05,
           9.5473e-06, 1.0863e-05],
          [1.4198e-05, 1.7930e-05, 1.9462e-05,  ..., 1.2362e-05,
           1.0356e-05, 1.2597e-05],
          [1.0847e-05, 1.3987e-05, 1.5398e-05,  ..., 1.4463e-05,
           1.2472e-05, 1.0189e-05]]]], device='cuda:0',
       grad_fn=<SigmoidBackward0>)

In [161]:
confidences.shape

torch.Size([1, 1, 256, 256])

In [160]:
confidences[0].shape

torch.Size([1, 256, 256])

In [159]:
confidences[0][0].shape

torch.Size([256, 256])

In [166]:
print(torch.round(confidences[0][0][0][0]))

tensor(0., device='cuda:0', grad_fn=<RoundBackward0>)


마스크 부분을 수정해야 제대로 된 결과가 나올 것으로 예상됨.

### * fasterrcnn

In [183]:
import torchvision
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.rpn import AnchorGenerator

backbone = torchvision.models.mobilenet_v2(pretrained=True).features
backbone.out_channels = 1280

anchor_generator = AnchorGenerator(sizes=((32, 64, 128, 256, 512),),
                                   aspect_ratios=((0.5, 1.0, 2.0),))

roi_pooler = torchvision.ops.MultiScaleRoIAlign(featmap_names=['0'],
                                                output_size=7,
                                                sampling_ratio=2)

model = FasterRCNN(backbone,
                   num_classes=5,
                   rpn_anchor_generator=anchor_generator,
                   box_roi_pool=roi_pooler)

model.eval()

Downloading: "https://download.pytorch.org/models/mobilenet_v2-b0353104.pth" to /root/.cache/torch/hub/checkpoints/mobilenet_v2-b0353104.pth


  0%|          | 0.00/13.6M [00:00<?, ?B/s]

FasterRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): Sequential(
    (0): ConvNormActivation(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU6(inplace=True)
    )
    (1): InvertedResidual(
      (conv): Sequential(
        (0): ConvNormActivation(
          (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
          (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (2): ReLU6(inplace=True)
        )
        (1): Conv2d(32, 16, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
    )
    (2): InvertedResidual(


In [184]:
train_dataset = CustomDataset(PATH, mode='train')
test_dataset = CustomDataset(PATH, mode='test')

train_idx = random.randint(0, train_dataset.__len__())
test_idx = random.randint(0, test_dataset.__len__())

print("\nTrain Datasets")
quick_check(train_dataset, train_idx)
print("\nTest Datasets")
quick_check(test_dataset, test_idx)

100%|██████████| 2/2 [00:00<00:00, 114.34it/s]



Train Datasets
1227번째 train 이미지
	 이미지 shape : torch.Size([3, 2048, 2048])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([3, 4])
			type : torch.FloatTensor
			tensor([[ 712.,  886., 1198., 1160.],
        [ 229.,  740.,  634., 1014.],
        [ 628.,  411.,  844.,  829.]])


		dict_keys == 'labels' :  
			shape : torch.Size([3])
			type : torch.LongTensor
			tensor([1, 1, 1])


		dict_keys == 'masks' :  
			shape : torch.Size([3, 2048, 2048])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[1227],
        [1227],
        [1227]])


		dict_keys == 'area' :  
			shape : torch.Size([3, 1])
			type : torch.FloatTensor
			tensor([[78723.5000],
        [65639.0000],
        [64169.5000]])


		dict_keys == 'iscrowd' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[0],
        [0],
        [0]])



Test Datasets
231번째 test 이미지
	 이미지 shape : torch.Size([3, 256,

In [185]:
img, _ = train_dataset.__getitem__(train_idx)
input_batch = img.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

In [186]:
output

[{'boxes': tensor([[2.1186e+01, 1.3385e+03, 1.0081e+03, 2.0480e+03],
          [2.6927e+02, 1.6911e+03, 3.8648e+02, 1.9292e+03],
          [1.8649e+02, 1.7027e+03, 3.0089e+02, 1.9160e+03],
          [3.4875e+02, 1.6951e+03, 4.6421e+02, 1.9312e+03],
          [1.2511e+03, 1.6949e+03, 1.3652e+03, 1.9278e+03],
          [1.1692e+03, 1.6949e+03, 1.2833e+03, 1.9277e+03],
          [1.0873e+03, 1.6948e+03, 1.2014e+03, 1.9277e+03],
          [1.0053e+03, 1.6948e+03, 1.1194e+03, 1.9277e+03],
          [9.2341e+02, 1.6948e+03, 1.0375e+03, 1.9277e+03],
          [1.3331e+03, 1.6949e+03, 1.4471e+03, 1.9278e+03],
          [8.4149e+02, 1.6948e+03, 9.5555e+02, 1.9277e+03],
          [4.3179e+02, 1.6953e+03, 5.4591e+02, 1.9284e+03],
          [1.4150e+03, 1.6949e+03, 1.5292e+03, 1.9278e+03],
          [7.5957e+02, 1.6948e+03, 8.7363e+02, 1.9277e+03],
          [5.9574e+02, 1.6948e+03, 7.0979e+02, 1.9277e+03],
          [6.7765e+02, 1.6948e+03, 7.9171e+02, 1.9277e+03],
          [5.1389e+02, 1.6947e+

In [191]:
print("Train Data Output")
uni_inference(output, train_idx)

Train Data Output
inference no.1227 : 
	boxes : 
		 shape : torch.Size([100, 4])

	labels : 
		 shape : torch.Size([100])

	scores : 
		 shape : torch.Size([100])



In [192]:
img, _ = test_dataset.__getitem__(test_idx)

input_batch = img.unsqueeze(0)

model.eval()
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

predictions = model(input_batch)

In [194]:
predictions

[{'boxes': tensor([[ 22.6541,  22.6714,  36.2568,  43.2240],
          [123.2007,   3.5830, 162.3059,  82.9571],
          [102.6815,   3.5628, 141.8291,  82.9354],
          [143.6196,   3.9919, 182.7745,  83.3374],
          [ 82.1691,   3.6955, 121.2818,  82.9451],
          [ 61.7119,   3.7490, 100.8140,  82.9393],
          [164.1181,   4.6507, 203.4202,  85.1843],
          [ 41.3099,   3.8039,  80.3901,  83.0411],
          [ 12.5812,   4.9828,  49.8068,  85.4255],
          [ 22.4761,  34.1992,  35.8650,  55.2395],
          [ 21.2685,  25.4661,  60.0878, 104.9288],
          [ 16.0646,  33.6116,  57.6379,  41.9813],
          [159.7459,  33.4850, 199.6217,  41.6733],
          [ 67.5610,  33.5238, 107.4440,  41.7073],
          [128.9939,  33.5610, 168.8706,  41.7357],
          [ 88.0315,  33.5099, 127.9200,  41.6989],
          [108.4872,  33.5417, 148.3909,  41.7241],
          [ 22.4386,  55.1692,  35.7900,  76.1828],
          [ 22.4528, 147.2960,  35.8036, 168.3235],
   

In [193]:
print("Test Data Output")
uni_inference(output, test_idx)

Test Data Output
inference no.231 : 
	boxes : 
		 shape : torch.Size([100, 4])

	labels : 
		 shape : torch.Size([100])

	scores : 
		 shape : torch.Size([100])



### * maskrcnn

In [195]:
import torchvision
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.models.detection.mask_rcnn import MaskRCNNPredictor
# MaskRCNN
# MaskRCNNHeads

def get_model_instance_segmentation(num_classes):
    model = torchvision.models.detection.maskrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)

    in_features_mask = model.roi_heads.mask_predictor.conv5_mask.in_channels
    hidden_layer = 256

    model.roi_heads.mask_predictor = MaskRCNNPredictor(in_features_mask,
                                                       hidden_layer,
                                                       num_classes)

    return model

In [197]:
model = get_model_instance_segmentation(num_classes=5)
model.eval()

Downloading: "https://download.pytorch.org/models/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth" to /root/.cache/torch/hub/checkpoints/maskrcnn_resnet50_fpn_coco-bf2d0c1e.pth


  0%|          | 0.00/170M [00:00<?, ?B/s]

MaskRCNN(
  (transform): GeneralizedRCNNTransform(
      Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
      Resize(min_size=(800,), max_size=1333, mode='bilinear')
  )
  (backbone): BackboneWithFPN(
    (body): IntermediateLayerGetter(
      (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): FrozenBatchNorm2d(64, eps=0.0)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): FrozenBatchNorm2d(64, eps=0.0)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): FrozenBatchNorm2d(64, eps=0.0)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): FrozenBatchNorm2d(256, eps=0.0)
          (relu): ReLU(in

In [198]:
train_dataset = CustomDataset(PATH, mode='train')
test_dataset = CustomDataset(PATH, mode='test')

train_idx = random.randint(0, train_dataset.__len__())
test_idx = random.randint(0, test_dataset.__len__())

print("\nTrain Datasets")
quick_check(train_dataset, train_idx)
print("\nTest Datasets")
quick_check(test_dataset, test_idx)

100%|██████████| 2/2 [00:00<00:00, 94.91it/s]



Train Datasets
1206번째 train 이미지
	 이미지 shape : torch.Size([3, 2048, 2048])
	 이미지 type : torch.FloatTensor
		dict_keys == 'boxes' :  
			shape : torch.Size([3, 4])
			type : torch.FloatTensor
			tensor([[ 832.,  665., 1191., 1009.],
        [ 395.,  729.,  763., 1082.],
        [ 716., 1129.,  961., 1612.]])


		dict_keys == 'labels' :  
			shape : torch.Size([3])
			type : torch.LongTensor
			tensor([1, 1, 1])


		dict_keys == 'masks' :  
			shape : torch.Size([3, 2048, 2048])
			type : torch.ByteTensor
		dict_keys == 'image_id' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[1206],
        [1206],
        [1206]])


		dict_keys == 'area' :  
			shape : torch.Size([3, 1])
			type : torch.FloatTensor
			tensor([[52780.],
        [51960.],
        [62293.]])


		dict_keys == 'iscrowd' :  
			shape : torch.Size([3, 1])
			type : torch.LongTensor
			tensor([[0],
        [0],
        [0]])



Test Datasets
241번째 test 이미지
	 이미지 shape : torch.Size([3, 256, 256])
	 이미지

In [199]:
img, _ = train_dataset.__getitem__(train_idx)
input_batch = img.unsqueeze(0)

if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

with torch.no_grad():
    output = model(input_batch)

In [200]:
output

[{'boxes': tensor([[5.7536e+00, 2.8919e+02, 6.8827e+01, 3.5940e+02],
          [5.5641e+00, 3.7109e+02, 6.9109e+01, 4.4187e+02],
          [5.5232e+00, 9.8553e+02, 6.9152e+01, 1.0564e+03],
          [5.5240e+00, 1.3337e+03, 6.9160e+01, 1.4046e+03],
          [5.5208e+00, 4.1208e+02, 6.9157e+01, 4.8296e+02],
          [5.5237e+00, 1.5385e+03, 6.9160e+01, 1.6094e+03],
          [5.5254e+00, 1.0265e+03, 6.9157e+01, 1.0974e+03],
          [5.5199e+00, 1.1084e+03, 6.9159e+01, 1.1793e+03],
          [5.5227e+00, 1.3951e+03, 6.9162e+01, 1.4660e+03],
          [5.5220e+00, 1.3644e+03, 6.9158e+01, 1.4353e+03],
          [5.5399e+00, 5.1449e+02, 6.9143e+01, 5.8536e+02],
          [5.5307e+00, 9.1385e+02, 6.9159e+01, 9.8473e+02],
          [5.5225e+00, 1.2313e+03, 6.9165e+01, 1.3022e+03],
          [5.5231e+00, 1.2927e+03, 6.9165e+01, 1.3636e+03],
          [5.5290e+00, 7.5001e+02, 6.9162e+01, 8.2089e+02],
          [5.5224e+00, 5.8617e+02, 6.9162e+01, 6.5704e+02],
          [5.5011e+00, 1.5999e+

In [202]:
print("Train Data Output")
uni_inference(output, train_idx)

Train Data Output
inference no.1206 : 
	boxes : 
		 shape : torch.Size([100, 4])

	labels : 
		 shape : torch.Size([100])

	scores : 
		 shape : torch.Size([100])

	masks : 
		 shape : torch.Size([100, 1, 2048, 2048])



In [203]:
img, _ = test_dataset.__getitem__(test_idx)

input_batch = img.unsqueeze(0)

model.eval()
if torch.cuda.is_available():
    input_batch = input_batch.to('cuda')
    model.to('cuda')

predictions = model(input_batch)

In [204]:
print("Test Data Output")
uni_inference(output, test_idx)

Test Data Output
inference no.241 : 
	boxes : 
		 shape : torch.Size([100, 4])

	labels : 
		 shape : torch.Size([100])

	scores : 
		 shape : torch.Size([100])

	masks : 
		 shape : torch.Size([100, 1, 2048, 2048])



## :construction: CustomModel :construction:

In [206]:
from torchvision import datasets
from torchvision.utils import make_grid
from torchvision.io import read_image
from torchvision import transforms as T

def get_transform(train):
    transforms = []
    transforms.append(T.ToTensor())
    if train:
        transforms.append(T.RandomHorizontalFlip(0.5))
    return T.Compose(transforms)


transform = T.Compose([
        T.Resize(256),
        T.RandomCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

train_transform = T.Compose([
        T.Resize(256),
        T.RandomCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

transform = T.Compose([
        T.Resize(256),
        T.RandomCrop(224),
        T.RandomHorizontalFlip(),
        T.ToTensor(),
        T.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]) 

train_dataset = datasets.ImageFolder(root=PATH+'/train/image', transform = train_transform)

train_dataset

Dataset ImageFolder
    Number of datapoints: 4000
    Root location: /content/drive/MyDrive/Task/plastic-segmentation/Data/train/image
    StandardTransform
Transform: Compose(
               Resize(size=256, interpolation=bilinear, max_size=None, antialias=None)
               RandomCrop(size=(224, 224), padding=None)
               RandomHorizontalFlip(p=0.5)
               ToTensor()
               Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
           )

In [215]:
# No changes tried with the optimizer yet.
MOMENTUM = 0.9
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0005

from typing import Any, Callable, cast, Dict, List, Optional, Tuple

train_dataset = CustomDataset(root=PATH, mode = 'train')
test_dataset = CustomDataset(root=PATH, mode='test')

# pe_trainset = torchvision.datasets.ImageFolder(root = f'{PATH}/image/PE', transform=)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=LEARNING_RATE, momentum=MOMENTUM, weight_decay=WEIGHT_DECAY)
#optimizer = torch.optim.Adam(params, lr=LEARNING_RATE, weight_decay=WEIGHT_DECAY)


from torch.utils.data import DataLoader
dl_train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, collate_fn=lambda x: tuple(zip(*x)))
dl_val = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=True, pin_memory=True, collate_fn=lambda x: tuple(zip(*x)))

lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

n_batches, n_batches_val = len(train_dataset), len(test_dataset)

validation_mask_losses = []

for epoch in range(1, NUM_EPOCHS + 1):
    print(f"Starting epoch {epoch} of {NUM_EPOCHS}")

    time_start = time.time()
    loss_accum = 0.0
    loss_mask_accum = 0.0
    loss_classifier_accum = 0.0
    # for batch_idx, (images, targets) in enumerate(dl_train, 1):
    for batch_idx, (images, targets) in enumerate(dl_train):
    
        # Predict
        images = list(image.to(DEVICE) for image in images)
        targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)[0]
        print(type(loss_dict), len(loss_dict))
        loss = sum(loss for loss in loss_dict.values())
        
        # Backprop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        # Logging
        loss_mask = loss_dict['loss_mask'].item()
        loss_accum += loss.item()
        loss_mask_accum += loss_mask
        loss_classifier_accum += loss_dict['loss_classifier'].item()
        
        if batch_idx % 500 == 0:
            print(f"    [Batch {batch_idx:3d} / {n_batches:3d}] Batch train loss: {loss.item():7.3f}. Mask-only loss: {loss_mask:7.3f}.")
                        
    if USE_SCHEDULER:
        lr_scheduler.step()

    # Train losses
    train_loss = loss_accum / n_batches
    train_loss_mask = loss_mask_accum / n_batches
    train_loss_classifier = loss_classifier_accum / n_batches

    # Validation
    val_loss_accum = 0
    val_loss_mask_accum = 0
    val_loss_classifier_accum = 0
    
    with torch.no_grad():
        for batch_idx, (images, targets) in enumerate(dl_val, 1):
            images = list(image.to(DEVICE) for image in images)
            targets = [{k: v.to(DEVICE) for k, v in t.items()} for t in targets]

            val_loss_dict = model(images, targets)
            val_batch_loss = sum(loss for loss in val_loss_dict.values())
            val_loss_accum += val_batch_loss.item()
            val_loss_mask_accum += val_loss_dict['loss_mask'].item()
            val_loss_classifier_accum += val_loss_dict['loss_classifier'].item()

    # Validation losses
    val_loss = val_loss_accum / n_batches_val
    val_loss_mask = val_loss_mask_accum / n_batches_val
    val_loss_classifier = val_loss_classifier_accum / n_batches_val
    elapsed = time.time() - time_start

    validation_mask_losses.append(val_loss_mask)

    torch.save(model.state_dict(), f"pytorch_model-e{epoch}.bin")
    prefix = f"[Epoch {epoch:2d} / {NUM_EPOCHS:2d}]"
    print(prefix)
    print(f"{prefix} Train mask-only loss: {train_loss_mask:7.3f}, classifier loss {train_loss_classifier:7.3f}")
    print(f"{prefix} Val mask-only loss  : {val_loss_mask:7.3f}, classifier loss {val_loss_classifier:7.3f}")
    print(prefix)
    print(f"{prefix} Train loss: {train_loss:7.3f}. Val loss: {val_loss:7.3f} [{elapsed:.0f} secs]")
    print(prefix)

100%|██████████| 2/2 [00:00<00:00, 115.73it/s]


Starting epoch 1 of 5
<class 'dict'> 4


RuntimeError: ignored

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt

import torchvision.transforms.functional as F
from pathlib import Path    # sys가 인식할 수 있도록 경로화 시킴???


plt.rcParams["savefig.bbox"] = 'tight'


def show(imgs):
    if not isinstance(imgs, list):
        imgs = [imgs]
    fix, axs = plt.subplots(ncols=len(imgs), squeeze=False)
    for i, img in enumerate(imgs):
        img = img.detach()
        img = F.to_pil_image(img)
        axs[0, i].imshow(np.asarray(img))
        axs[0, i].set(xticklabels=[], yticklabels=[], xticks=[], yticks=[])

# additional refs

## utilites

In [95]:
# ref: https://www.kaggle.com/inversion/run-length-decoding-quick-start
def rle_decode(mask_rle, shape, color=1):
    '''
    mask_rle: run-length as string formated (start length)
    shape: (height, width, channels) of array to return
    color: color for the mask
    Returns numpy array (mask)

    '''
    s = mask_rle.split()

    starts = list(map(lambda x: int(x) - 1, s[0::2]))
    lengths = list(map(int, s[1::2]))
    ends = [x + y for x, y in zip(starts, lengths)]
    if len(shape)==3:
        img = np.zeros((shape[0] * shape[1], shape[2]), dtype=np.float32)
    else:
        img = np.zeros(shape[0] * shape[1], dtype=np.float32)
    for start, end in zip(starts, ends):
        img[start : end] = color

    return img.reshape(shape)


def rle_encoding(x):
    dots = np.where(x.flatten() == 1)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if (b>prev+1): run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return ' '.join(map(str, run_lengths))


def remove_overlapping_pixels(mask, other_masks):
    for other_mask in other_masks:
        if np.sum(np.logical_and(mask, other_mask)) > 0:
            mask[np.logical_and(mask, other_mask)] = 0
    return mask

def combine_masks(masks, mask_threshold):
    """
    combine masks into one image
    """
    maskimg = np.zeros((HEIGHT, WIDTH))
    # print(len(masks.shape), masks.shape)
    for m, mask in enumerate(masks,1):
        maskimg[mask>mask_threshold] = m
    return maskimg


def get_filtered_masks(pred):
    """
    filter masks using MIN_SCORE for mask and MAX_THRESHOLD for pixels
    """
    use_masks = []   
    for i, mask in enumerate(pred["masks"]):

        # Filter-out low-scoring results. Not tried yet.
        scr = pred["scores"][i].cpu().item()
        label = pred["labels"][i].cpu().item()
        if scr > min_score_dict[label]:
            mask = mask.cpu().numpy().squeeze()
            # Keep only highly likely pixels
            binary_mask = mask > mask_threshold_dict[label]
            binary_mask = remove_overlapping_pixels(binary_mask, use_masks)
            use_masks.append(binary_mask)

    return use_masks

## Metric

In [ ]:
# Metric: mean of the precision values at each IoU threshold?
# Ref: https://www.kaggle.com/theoviel/competition-metric-map-iou

def compute_iou(labels, y_pred, verbose=0):
    """
    Computes the IoU for instance labels and predictions.

    Args:
        labels (np array): Labels.
        y_pred (np array): predictions

    Returns:
        np array: IoU matrix, of size true_objects x pred_objects.
    """

    true_objects = len(np.unique(labels))
    pred_objects = len(np.unique(y_pred))

    if verbose:
        print("Number of true objects: {}".format(true_objects))
        print("Number of predicted objects: {}".format(pred_objects))

    # Compute intersection between all objects
    intersection = np.histogram2d(
        labels.flatten(), y_pred.flatten(), bins=(true_objects, pred_objects)
    )[0]

    # Compute areas (needed for finding the union between all objects)
    area_true = np.histogram(labels, bins=true_objects)[0]
    area_pred = np.histogram(y_pred, bins=pred_objects)[0]
    area_true = np.expand_dims(area_true, -1)
    area_pred = np.expand_dims(area_pred, 0)

    # Compute union
    union = area_true + area_pred - intersection
    intersection = intersection[1:, 1:] # exclude background
    union = union[1:, 1:]
    union[union == 0] = 1e-9
    iou = intersection / union
    
    return iou  

def precision_at(threshold, iou):
    """
    Computes the precision at a given threshold.

    Args:
        threshold (float): Threshold.
        iou (np array): IoU matrix.

    Returns:
        int: Number of true positives,
        int: Number of false positives,
        int: Number of false negatives.
    """
    matches = iou > threshold
    true_positives = np.sum(matches, axis=1) == 1  # Correct objects
    false_positives = np.sum(matches, axis=0) == 0  # Missed objects
    false_negatives = np.sum(matches, axis=1) == 0  # Extra objects
    tp, fp, fn = (
        np.sum(true_positives),
        np.sum(false_positives),
        np.sum(false_negatives),
    )
    return tp, fp, fn

def iou_map(truths, preds, verbose=0):
    """
    Computes the metric for the competition.
    Masks contain the segmented pixels where each object has one value associated,
    and 0 is the background.

    Args:
        truths (list of masks): Ground truths.
        preds (list of masks): Predictions.
        verbose (int, optional): Whether to print infos. Defaults to 0.

    Returns:
        float: mAP.
    """
    ious = [compute_iou(truth, pred, verbose) for truth, pred in zip(truths, preds)]

    if verbose:
        print("Thresh\tTP\tFP\tFN\tPrec.")

    prec = []
    for t in np.arange(0.5, 1.0, 0.05):
        tps, fps, fns = 0, 0, 0
        for iou in ious:
            tp, fp, fn = precision_at(t, iou)
            tps += tp
            fps += fp
            fns += fn

        p = tps / (tps + fps + fns)
        prec.append(p)

        if verbose:
            print("{:1.3f}\t{}\t{}\t{}\t{:1.3f}".format(t, tps, fps, fns, p))

    if verbose:
        print("AP\t-\t-\t-\t{:1.3f}".format(np.mean(prec)))

    return np.mean(prec)


def get_score(ds, mdl):
    """
    Get average IOU mAP score for a dataset
    """
    mdl.eval()
    iouscore = 0
    for i in tqdm(range(len(ds))):
        img, targets = ds[i]
        with torch.no_grad():
            result = mdl([img.to(DEVICE)])[0]
            
        masks = combine_masks(targets['masks'], 0.5)
        labels = pd.Series(result['labels'].cpu().numpy()).value_counts()

        mask_threshold = mask_threshold_dict[labels.sort_values().index[-1]]
        pred_masks = combine_masks(get_filtered_masks(result), mask_threshold)
        iouscore += iou_map([masks],[pred_masks])
    return iouscore / len(ds)

## augmentation_ver1

* These are slight redefinitions of torch.transformation classes
* The difference is that they handle the target and the mask
* Copied from Abishek, added new ones
* ref : https://www.kaggle.com/rluethy/sartorius-torch-mask-r-cnn

In [129]:
class Compose:
    def __init__(self, transforms):
        self.transforms = transforms

    def __call__(self, image, target):
        for t in self.transforms:
            image, target = t(image, target)
        return image, target

class VerticalFlip:
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-2)
            bbox = target["boxes"]
            bbox[:, [1, 3]] = height - bbox[:, [3, 1]]
            target["boxes"] = bbox
            target["masks"] = target["masks"].flip(-2)
        return image, target

class HorizontalFlip:
    def __init__(self, prob):
        self.prob = prob

    def __call__(self, image, target):
        if random.random() < self.prob:
            height, width = image.shape[-2:]
            image = image.flip(-1)
            bbox = target["boxes"]
            bbox[:, [0, 2]] = width - bbox[:, [2, 0]]
            target["boxes"] = bbox
            target["masks"] = target["masks"].flip(-1)
        return image, target

class Normalize:
    def __call__(self, image, target):
        image = F.normalize(image, RESNET_MEAN, RESNET_STD)
        return image, target

class ToTensor:
    def __call__(self, image, target):
        image = F.to_tensor(image)
        return image, target
    

def get_transform(train):
    transforms = [ToTensor()]
    if NORMALIZE:
        transforms.append(Normalize())
    
    # Data augmentation for train
    if train: 
        transforms.append(HorizontalFlip(0.5))
        transforms.append(VerticalFlip(0.5))

    return Compose(transforms)